# Titanic - Machine Learning from Disaster
Este notebook resolve o desafio de prever a sobrevivência dos passageiros do Titanic, utilizando aprendizado de máquina supervisionado com os dados disponíveis no Kaggle.

Este Notebook foi criado a partir do desafio Titanic do Kaggle  -  https://www.kaggle.com/competitions/titanic

Esta primeira célula é padrão dos notebooks dentro do kaggle.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


## Carregamento dos dados
Carregamento dos arquivos CSV de treino, teste e exemplo de submissão disponibilizados pela competição.

In [ ]:
train = pd.read_csv("/kaggle/input/titanic/train.csv")
test = pd.read_csv("/kaggle/input/titanic/test.csv")
submission_example = pd.read_csv("/kaggle/input/titanic/gender_submission.csv")

train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Pré-processamento dos dados
Preenchimento de valores nulos, codificação de variáveis categóricas e seleção de features relevantes.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

# Copiar os dados para não alterar o original
df = train.copy()

df['Age'] = df['Age'].fillna(df['Age'].median())
df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])


# Variáveis categóricas
df['Sex'] = LabelEncoder().fit_transform(df['Sex'])  # male=1, female=0
df['Embarked'] = LabelEncoder().fit_transform(df['Embarked'])  # S=2, C=0, Q=1 (ordem pode variar)

# Selecionar features
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
X = df[features]
y = df['Survived']

# Separar treino e validação
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinamento do modelo
modelo = LogisticRegression(max_iter=1000)
modelo.fit(X_train, y_train)

# Avaliação
y_pred = modelo.predict(X_val)
acc = accuracy_score(y_val, y_pred)
print("Acurácia do modelo de baseline:", acc)


Acurácia do modelo de baseline: 0.8100558659217877


## Treinamento do modelo
Utilizei Regressão Logística como modelo de baseline. A divisão entre treino e validação é de 80/20.

In [ ]:
# Mesmo pré-processamento no dataset de teste
test_df = test.copy()
test_df['Age'] = test_df['Age'].fillna(df['Age'].median())
test_df['Fare'] = test_df['Fare'].fillna(df['Fare'].median())
test_df['Sex'] = LabelEncoder().fit_transform(test_df['Sex'])
test_df['Embarked'] = LabelEncoder().fit_transform(test_df['Embarked'].fillna('S'))

# Mesmas features usadas no treino
X_test = test_df[features]

# Previsões
predictions = modelo.predict(X_test)

# Arquivo de submissão
submission = pd.DataFrame({
    "PassengerId": test_df["PassengerId"],
    "Survived": predictions
})

# Salvar CSV
submission.to_csv("submission.csv", index=False)


## Conclusão
A acurácia do modelo de baseline com Regressão Logística foi satisfatória, atingindo cerca de 81%. Esse notebook serve como ponto de partida para melhorias, incluindo EDA mais completa, uso de outros modelos e otimização de hiperparâmetros.